# Settign up

## Conexión a google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd ./drive/MyDrive/Colab Notebooks/

[Errno 2] No such file or directory: './drive/MyDrive/Colab Notebooks/'
/content/drive/MyDrive/Colab Notebooks


In [ ]:
%pwd

'/content/drive/My Drive/Colab Notebooks'

## Instalar microTC

In [ ]:
%pip install git+https://github.com/felipeRmBr/microtc.git#egg=microtc

## Importar módulos *_utils

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from my_utils import dataset_utils
from my_utils import eval_utils
from my_utils import svc_utils
from my_utils import results_utils

import pickle

## Importar el dataset

In [ ]:
X_train, Y_train = dataset_utils.importTrainDataForSVM()

# Experimentos con combinaciones de tokens

Ahora nos concentramos en ubicar las combinaciones de tokens asociadas a los mejores clasificadores. Haremos experimentos independientes para cada tarea, y para cada tipo de kernel elegiremos las cinco mejores combinaciones de tokens de acuerdo con los resultados de exactitud o EMR obtenidos en los experimentos.

### Variaciones de los modelos de texto

In [ ]:
# Define a list of tokens combos - 
from itertools import combinations

tokens_combos =[]

base_tokens_list = [-1,3]
candidate_tokens = [-3,-2,4,5,6,7]

for i in range(0,4):
  for combo in combinations(candidate_tokens, i):
    tokens_combos.append(sorted(base_tokens_list + list(combo)))

print('{} tokens combinations.'.format(len(tokens_combos)))

42 tokens combinations.


In [ ]:
# Define a  params dictionary -
# we focus on experimenting with the different tokens combos
# so we keep fixed the rest of the parameters

from itertools import product
text_model_params = {'num_option':['group'],
                     'usr_option':['group'],
                     'url_option':['group'],
                     'hashtag_option':['group'],
                     'del_punc':[True],
                     'del_dup':[True],
                     'del_diac':[True],
                     'token_list': tokens_combos,
                     'weighting':['tfidf']
                     }

text_model_search_space = list(product(*text_model_params.values()))
print(len(text_model_search_space), ' combinations')

42  combinations


### SVM variations

In [ ]:
svm_params = {'kernel':['linear', 'sigmoid'],
              'C':[2]}

svm_search_space = list(product(*svm_params.values()))
print(len(svm_search_space), ' combinations')           

2  combinations


### Scanning

In [ ]:
# Task = 'HS
df = svc_utils.svcSpaceScanner(X_train, Y_train,
                    'HS',
                    text_model_search_space,
                    list(text_model_params.keys()),
                    svm_search_space,
                    list(svm_params.keys()),
                    n_folds=7,
                    metrics=['acc'],
                    partial_CV=True,
                    backup_file='./Results/SVC/phase-2/HS.df')

In [ ]:
# Task = 'TR'
df = svc_utils.svcSpaceScanner(X_train, Y_train,
                    'TR',
                    text_model_search_space[21:],
                    list(text_model_params.keys()),
                    svm_search_space,
                    list(svm_params.keys()),
                    n_folds=7,
                    metrics=['acc'],
                    partial_CV=True,
                    backup_file='./Results/SVC/phase-2/TR.df')

In [ ]:
# Task = 'AG'
df = svc_utils.svcSpaceScanner(X_train, Y_train,
                    'AG',
                    text_model_search_space[21:],
                    list(text_model_params.keys()),
                    svm_search_space,
                    list(svm_params.keys()),
                    n_folds=7,
                    metrics=['acc'],
                    partial_CV=True,
                    backup_file='./Results/SVC/phase-2/AG.df')

In [ ]:
# Task  'AG'
df = svc_utils.svcSpaceScanner(X_train, Y_train,
                    'HTA',
                    text_model_search_space,
                    list(text_model_params.keys()),
                    svm_search_space,
                    list(svm_params.keys()),
                    n_folds=7,
                    metrics=['acc'],
                    partial_CV=True,
                    backup_file='./Results/SVC/phase-2/HTA.df')

# Resultados

### Tarea A ('HS')

In [ ]:
import pickle
with open('./Results/SVC/phase-2/HS.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [ ]:
results_df.head()

**linear kernel**

In [ ]:
results_subset = results_df.loc[results_df['kernel']=='linear']

results_subset.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_subset.reset_index(drop=True, inplace=True)
columns2show = ['kernel', 'token_list', 'mean_acc', 'mean_val_acc']
results_subset.loc[:4,columns2show]

In [ ]:
results_subset[:5]['token_list'].to_list()

[[-3, -2, -1, 3, 7],
 [-2, -1, 3, 7],
 [-2, -1, 3, 5, 7],
 [-3, -2, -1, 3, 6],
 [-3, -1, 3, 5, 7]]

**sigmoid kernel**

In [ ]:
results_subset = results_df.loc[results_df['kernel']=='sigmoid']

results_subset.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_subset.reset_index(drop=True, inplace=True)
columns2show = ['kernel', 'token_list', 'mean_acc', 'mean_val_acc']
results_subset.loc[:4,columns2show]

In [ ]:
results_subset[:5]['token_list'].to_list()

[[-3, -2, -1, 3, 7],
 [-2, -1, 3, 7],
 [-3, -1, 3, 7],
 [-3, -2, -1, 3, 6],
 [-2, -1, 3, 6, 7]]

### Tarea B1 ('TR')

In [ ]:
import pickle
with open('./Results/SVC/phase-2/TR.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [ ]:
results_df.head()

,num_option,usr_option,url_option,hashtag_option,del_punc,del_dup,del_diac,token_list,weighting,kernel,C,mean_acc,mean_val_acc,mean_f1,mean_val_f1
0,group,group,group,group,True,True,True,"[-1, 3]",tfidf,linear,2,0.996850,0.860001,0.995826,0.799982
1,group,group,group,group,True,True,True,"[-1, 3]",tfidf,sigmoid,2,0.976198,0.861110,0.967756,0.801407
2,group,group,group,group,True,True,True,"[-3, -1, 3]",tfidf,linear,2,0.998950,0.870151,0.998612,0.815824
3,group,group,group,group,True,True,True,"[-3, -1, 3]",tfidf,sigmoid,2,0.996500,0.869918,0.995353,0.817062
4,group,group,group,group,True,True,True,"[-2, -1, 3]",tfidf,linear,2,0.998950,0.869801,0.998612,0.814724


**linear kernel**

In [ ]:
results_subset = results_df.loc[results_df['kernel']=='linear']

results_subset.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_subset.reset_index(drop=True, inplace=True)
columns2show = ['kernel', 'token_list', 'mean_acc', 'mean_val_acc', 'mean_f1', 'mean_val_f1']
results_subset.loc[:4,columns2show]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,kernel,token_list,mean_acc,mean_val_acc,mean_f1,mean_val_f1
0,linear,"[-3, -2, -1, 3, 6]",0.9993,0.874993,0.999075,0.818852
1,linear,"[-1, 3, 7]",0.9993,0.874934,0.999075,0.819356
2,linear,"[-3, -1, 3, 7]",0.9993,0.874818,0.999075,0.818965
3,linear,"[-2, -1, 3, 6]",0.9993,0.874468,0.999075,0.818319
4,linear,"[-3, -2, -1, 3, 7]",0.9993,0.874409,0.999075,0.818126


In [ ]:
results_subset[:5]['token_list'].to_list()

[[-3, -2, -1, 3, 6],
 [-1, 3, 7],
 [-3, -1, 3, 7],
 [-2, -1, 3, 6],
 [-3, -2, -1, 3, 7]]

**sigmoid kernel**

In [ ]:
results_subset = results_df.loc[results_df['kernel']=='sigmoid']

results_subset.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_subset.reset_index(drop=True, inplace=True)
columns2show = ['kernel', 'token_list', 'mean_acc', 'mean_val_acc', 'mean_f1', 'mean_val_f1']
results_subset.loc[:4,columns2show]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,kernel,token_list,mean_acc,mean_val_acc,mean_f1,mean_val_f1
0,sigmoid,"[-3, -1, 3, 7]",0.9986,0.878493,0.998152,0.827787
1,sigmoid,"[-3, -2, -1, 3, 6]",0.9986,0.878493,0.998152,0.827555
2,sigmoid,"[-3, -2, -1, 3, 7]",0.9986,0.878084,0.998152,0.827324
3,sigmoid,"[-3, -1, 3, 6, 7]",0.9986,0.877851,0.998152,0.826117
4,sigmoid,"[-2, -1, 3, 5, 7]",0.9986,0.877734,0.998152,0.825851


In [ ]:
results_subset.loc[37:,columns2show]

,kernel,token_list,mean_acc,mean_val_acc,mean_f1,mean_val_f1
37,sigmoid,"[-3, -1, 3]",0.996500,0.869918,0.995353,0.817062
38,sigmoid,"[-2, -1, 3, 4]",0.993350,0.869451,0.991153,0.813724
39,sigmoid,"[-1, 3, 4, 5]",0.995100,0.868693,0.993490,0.810984
40,sigmoid,"[-1, 3, 4]",0.985650,0.864434,0.980746,0.805288
41,sigmoid,"[-1, 3]",0.976198,0.861110,0.967756,0.801407


In [ ]:
results_subset[:5]['token_list'].to_list()

[[-3, -1, 3, 7],
 [-3, -2, -1, 3, 6],
 [-3, -2, -1, 3, 7],
 [-3, -1, 3, 6, 7],
 [-2, -1, 3, 5, 7]]

### Tarea B2 ('AG')

In [ ]:
import pickle
with open('./Results/SVC/phase-2/AG.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [ ]:
results_df.head()

,num_option,usr_option,url_option,hashtag_option,del_punc,del_dup,del_diac,token_list,weighting,kernel,C,mean_acc,mean_val_acc,mean_f1,mean_val_f1
0,group,group,group,group,True,True,True,"[-1, 3]",tfidf,linear,2,0.996850,0.860001,0.995826,0.799982
1,group,group,group,group,True,True,True,"[-1, 3]",tfidf,sigmoid,2,0.976198,0.861110,0.967756,0.801407
2,group,group,group,group,True,True,True,"[-3, -1, 3]",tfidf,linear,2,0.998950,0.870151,0.998612,0.815824
3,group,group,group,group,True,True,True,"[-3, -1, 3]",tfidf,sigmoid,2,0.996500,0.869918,0.995353,0.817062
4,group,group,group,group,True,True,True,"[-2, -1, 3]",tfidf,linear,2,0.998950,0.869801,0.998612,0.814724


**linear kernel**

In [ ]:
results_subset = results_df.loc[results_df['kernel']=='linear']

results_subset.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_subset.reset_index(drop=True, inplace=True)
columns2show = ['kernel', 'token_list', 'mean_acc', 'mean_val_acc', 'mean_f1', 'mean_val_f1']
results_subset.loc[:4,columns2show]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,kernel,token_list,mean_acc,mean_val_acc,mean_f1,mean_val_f1
0,linear,"[-3, -2, -1, 3, 6]",0.9993,0.874993,0.999075,0.818852
1,linear,"[-1, 3, 7]",0.9993,0.874934,0.999075,0.819356
2,linear,"[-3, -1, 3, 7]",0.9993,0.874818,0.999075,0.818965
3,linear,"[-2, -1, 3, 6]",0.9993,0.874468,0.999075,0.818319
4,linear,"[-3, -2, -1, 3, 7]",0.9993,0.874409,0.999075,0.818126


In [ ]:
results_subset[:5]['token_list'].to_list()

[[-3, -2, -1, 3, 6],
 [-1, 3, 7],
 [-3, -1, 3, 7],
 [-2, -1, 3, 6],
 [-3, -2, -1, 3, 7]]

**sigmoid kernel**

In [ ]:
results_subset = results_df.loc[results_df['kernel']=='sigmoid']

results_subset.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_subset.reset_index(drop=True, inplace=True)
columns2show = ['kernel', 'token_list', 'mean_acc', 'mean_val_acc', 'mean_f1', 'mean_val_f1']
results_subset.loc[:4,columns2show]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,kernel,token_list,mean_acc,mean_val_acc,mean_f1,mean_val_f1
0,sigmoid,"[-3, -1, 3, 7]",0.9986,0.878493,0.998152,0.827787
1,sigmoid,"[-3, -2, -1, 3, 6]",0.9986,0.878493,0.998152,0.827555
2,sigmoid,"[-3, -2, -1, 3, 7]",0.9986,0.878084,0.998152,0.827324
3,sigmoid,"[-3, -1, 3, 6, 7]",0.9986,0.877851,0.998152,0.826117
4,sigmoid,"[-2, -1, 3, 5, 7]",0.9986,0.877734,0.998152,0.825851


In [ ]:
results_subset.loc[37:,columns2show]

,kernel,token_list,mean_acc,mean_val_acc,mean_f1,mean_val_f1
37,sigmoid,"[-3, -1, 3]",0.996500,0.869918,0.995353,0.817062
38,sigmoid,"[-2, -1, 3, 4]",0.993350,0.869451,0.991153,0.813724
39,sigmoid,"[-1, 3, 4, 5]",0.995100,0.868693,0.993490,0.810984
40,sigmoid,"[-1, 3, 4]",0.985650,0.864434,0.980746,0.805288
41,sigmoid,"[-1, 3]",0.976198,0.861110,0.967756,0.801407


In [ ]:
results_subset[:5]['token_list'].to_list()

[[-3, -1, 3, 7],
 [-3, -2, -1, 3, 6],
 [-3, -2, -1, 3, 7],
 [-3, -1, 3, 6, 7],
 [-2, -1, 3, 5, 7]]

### Tarea conjunta ('HTA')

In [ ]:
import pickle
with open('./Results/SVC/phase-2/HTA.df', 'rb') as file_handler:
  results_df = pickle.load(file_handler)

In [ ]:
results_df.head()

**linear kernel**

In [ ]:
results_subset = results_df.loc[results_df['kernel']=='linear']

In [ ]:
results_subset.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_subset.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
columns2show = ['kernel', 'token_list', 'mean_acc', 'mean_val_acc']
results_subset.loc[:4,columns2show]

,kernel,token_list,mean_acc,mean_val_acc
0,linear,"[-3, -2, -1, 3]",0.999300,0.729277
1,linear,"[-3, -2, -1, 3, 4]",0.998951,0.727585
2,linear,"[-2, -1, 3]",0.998600,0.727235
3,linear,"[-3, -1, 3]",0.998251,0.727002
4,linear,"[-2, -1, 3, 4, 7]",0.998951,0.726886


In [ ]:
results_subset[:5]['token_list'].to_list()

[[-3, -2, -1, 3],
 [-3, -2, -1, 3, 4],
 [-2, -1, 3],
 [-3, -1, 3],
 [-2, -1, 3, 4, 7]]

**sigmoid kernel**

In [ ]:
results_subset = results_df.loc[results_df['kernel']=='sigmoid']

In [ ]:
results_subset.sort_values(by='mean_val_acc', ascending=False, inplace=True)
results_subset.reset_index(drop=True, inplace=True)
columns2show = ['kernel', 'token_list', 'mean_acc', 'mean_val_acc']
results_subset.loc[:4,columns2show]

,kernel,token_list,mean_acc,mean_val_acc
0,sigmoid,"[-3, -1, 3, 7]",0.996851,0.733710
1,sigmoid,"[-1, 3, 7]",0.996851,0.733419
2,sigmoid,"[-3, -2, -1, 3, 6]",0.996851,0.733302
3,sigmoid,"[-3, -1, 3, 6]",0.996851,0.732835
4,sigmoid,"[-3, -2, -1, 3, 5]",0.996851,0.732544


In [ ]:
results_subset[:5]['token_list'].to_list()

[[-3, -1, 3, 7],
 [-1, 3, 7],
 [-3, -2, -1, 3, 6],
 [-3, -1, 3, 6],
 [-3, -2, -1, 3, 5]]